# 1.  Install Necessary Package

In [1]:
# This could be removed after installed

import subprocess
import sys

# Install xlwings
def install_xlwings():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "xlwings"])

try:
    import xlwings as xw
except ModuleNotFoundError:
    print("xlwings is installing...")
    install_xlwings()
    import xlwings as xw


# 2. Choose Entity if there is duplicate Code

In [2]:
import os
import glob
import xlwings as xw

# Get the current working directory
script_dir = os.getcwd()

# Search for the first Excel file in the directory (adjust extension if needed)
excel_files = glob.glob(os.path.join(script_dir, '*.xlsm'))  # Finds .xls, .xlsx, .xlsm files

if excel_files:
    # Use the first Excel file found
    excel_path = excel_files[0]
    # Extract the file name from the path
    excel_name = os.path.basename(excel_path)
    print(f"Excel file name: {excel_name}")
    
    print(f"Opening Excel file: {excel_path}")
    
    # Open Excel in the background and open the workbook
    app = xw.App(visible=False)
    wb = app.books.open(excel_path)
    
    # List of sheet names you want to check
    sheet_names = ['File Path']  # Add sheet names as needed

    # Loop through each sheet and get the values of B1 and A13
    for sheet_name in sheet_names:
        sheet = wb.sheets[sheet_name]
        B1_value = sheet.range('B1').value
        A13_value = sheet.range('A13').value
        print(f"Sheet '{sheet_name}' - Value of B1: {B1_value}, Value of A13: {A13_value}")

    # Close the workbook and quit the app
    wb.close()
    app.quit()
else:
    print("No Excel files found in the current directory.")


Excel file name: YEAR Annual Workpapers.xlsm
Opening Excel file: c:\Users\Jiawei Qi\Documents\Intern\4. Upload\Python\20241126\YEAR Annual Workpapers.xlsm
Sheet 'File Path' - Value of B1: C:\Users\Jiawei Qi\Documents\Intern\test\2025, Value of A13: testac


In [3]:
import requests
import json
import tkinter as tk
from tkinter import ttk, messagebox

# Define API request constants
url = "https://api-ap-southeast-2.fyi.app/external/entity"
ACCESS_KEY = '56c2e94d-ceaa-496a-bdcd-aa101b5731a7'
SECRET_KEY = '414643fd63bad5b9f6028dc68dfab3180be5c26d95970e68025371eddef28aca'
NAME = A13_value  # Replace with actual value or automatically generate it from the filepath.

# Prepare the API request payload
payload = {
    "metadata": {
        "action": {
            "value": "list"
        },
        "data": {
            "name": NAME,
            "page": 1,
            "per_page": 10
        }
    }
}

headers = {
    'x-fyi-access-id': ACCESS_KEY,
    'x-fyi-access-secret': SECRET_KEY,
    'Content-Type': 'application/json'
}

# Send the POST request to the API
response = requests.post(url, headers=headers, json=payload)

# Check if the request was successful and contains results
if response.status_code == 200:
    data = response.json()
    
    if 'results' in data and data['results']:
        # Filter results to include only entities with a name that matches k2_value
        matching_entities = [
            result for result in data['results']
            if result.get('name') == NAME
        ]
        
        # Check if there are matching entities
        if matching_entities:
            # Extract client codes and IDs for matching entities
            client_options = {
                f"{result['details']['client_code']} - {result['name']}": result['id']
                for result in matching_entities
                if 'details' in result and 'client_code' in result['details']
            }
            
            # If only one matching entity is available, use it directly
            if len(client_options) == 1:
                selected_text, selected_id = next(iter(client_options.items()))
                print(f"Only one matching entity found. Automatically selected:\nClient: {selected_text}\nID: {selected_id}")
                ENTITY_ID = selected_id
                # Proceed with the selected ID for further processing
                # Add additional processing here with selected_id
                
            else:
                # Initialize tkinter window for selection
                root = tk.Tk()
                root.title("Select Client Code")
                
                # Define a variable to store the selected ID
                root.selected_id = None  # Initialize it as None

                # Function to handle client code selection
                def on_select(event):
                    selected_text = client_code_var.get()
                    root.selected_id = client_options[selected_text]  # Store the selected ID
                    messagebox.showinfo("Selection", f"Selected Client: {selected_text}\nCorresponding ID: {root.selected_id}")
                    ENTITY_ID = root.selected_id
                    root.destroy()  # Close the tkinter window after selection
                
                # Label for dropdown
                tk.Label(root, text="Please select a client:").pack(pady=10)

                # Dropdown menu for client codes
                client_code_var = tk.StringVar()
                client_code_dropdown = ttk.Combobox(root, textvariable=client_code_var, values=list(client_options.keys()))
                client_code_dropdown.pack(pady=5)

                # Bind selection event
                client_code_dropdown.bind("<<ComboboxSelected>>", on_select)

                # Start the GUI loop
                root.mainloop()
                
                # After the window closes, access selected_id for further processing
                if root.selected_id:
                    print(f"Selected ID for further processing: {root.selected_id}")
                    ENTITY_ID = root.selected_id
                    # Add additional processing here with root.selected_id
                    
        else:
            print(f"No entities found with the name matching '{NAME}'.")
            
    else:
        print("No client codes available in the results.")
else:
    print(f"Request failed with status code {response.status_code}")

print(ENTITY_ID)


Only one matching entity found. Automatically selected:
Client: TESTAC - testac
ID: 34493554
34493554


# 3. Upload Document in FYI

In [4]:
import os
from pathlib import Path
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder

# Define the mapping between file extensions and the categories
file_type_mapping = {
    '.ppt': 'Presentation-Legacy',
    '.pptx': 'Presentation',
    '.eml': 'Email',
    '.msg': 'Email',
    '.jpg': 'Image',
    '.jpeg': 'Image',
    '.png': 'Image',
    '.gif': 'Image',
    '.bmp': 'Image',
    '.pdf': 'Pdf',
    '.xls': 'Spreadsheet',
    '.xlsx': 'Spreadsheet',
    '.xlsm': 'Spreadsheet',
    '.xltm': 'Spreadsheet',
    '.csv': 'CSV',
    '.doc': 'Letter-Legacy',
    '.docx': 'Letter',
    '.mp3': 'Phone',
    '.wav': 'Phone',
    '.txt': 'Text',
    '.zip': 'Zipped',
    '.rar': 'Zipped',
    '.7z': 'Zipped',
    '.template': 'Template',
    '.ics': 'Meeting',
    '.log': 'Note',
    '.html': 'Link',
    '.url': 'Link',
    # Add more extensions if necessary
}

# Default category for any file extension not listed
default_category = 'Other'

# Directory to scan
folder_path = Path(B1_value) / str(A13_value)

# Dictionary to store file paths and their corresponding categories
file_category_dict = {}

# Recursively get all file paths
for file in folder_path.glob('**/*'):
    if file.is_file():  # Only process files
        file_extension = file.suffix.lower()  # Convert extension to lowercase to handle case sensitivity
        file_category = file_type_mapping.get(file_extension, default_category)
        file_category_dict[str(file)] = file_category
print(file_category_dict)

# FYI API credentials and URL
ACCESS_KEY = '56c2e94d-ceaa-496a-bdcd-aa101b5731a7'
SECRET_KEY = '414643fd63bad5b9f6028dc68dfab3180be5c26d95970e68025371eddef28aca'
URL = "api-ap-southeast-2.fyi.app"
# ENTITY_ID =  root.selected_id

links=[]
def add_document_record(file_path, document_type,include_preview=True):
    headers = {
        'x-fyi-access-id': ACCESS_KEY,
        'x-fyi-access-secret': SECRET_KEY,
        'Accept': 'application/json',
        'Content-Type': 'application/json'
    }
    body = json.dumps({
        "metadata": {
            "action": {"value": 'upsert'},
            "data": {
                "model": {
                    "name": os.path.basename(file_path),
                    "document_type": document_type,
                    "entity_id": ENTITY_ID,
                    "original_filename":os.path.basename(file_path)
                }
            }
        }
    })
    
    response = requests.post(f"https://{URL}/external/document", headers=headers, data=body)

    if response.status_code == 200:
        response_json = response.json()
        if "data" in response_json and "version_id" in response_json["data"]:
            document_id=response_json["data"]["id"]
            # Conditionally include "preview" in the hyperlink
            if include_preview:
                hyperlink = f"https://go.fyi.app/search/0/{ENTITY_ID}/0/documents/{document_id}/preview"
            else:
                hyperlink = f"https://go.fyi.app/search/0/{ENTITY_ID}/0/documents/{document_id}"
            print(document_id)
            print(hyperlink)
            links.append(hyperlink)
            return response_json["data"]["version_id"]
        else:
            print(f"Error: Document version_id not found in response - {response_json}")
            return None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None
    

def authorise_upload(doc_id):
    headers = {
        'x-fyi-access-id': ACCESS_KEY,
        'x-fyi-access-secret': SECRET_KEY,
        'Accept': 'application/json',
        'Content-Type': 'application/json'
    }
    body = json.dumps({
        "metadata": {
            "action": {
                "value": 'uploadForm'
            },
            "data": {
                "id": doc_id
            }
        }
    })
    
    response = requests.post(f"https://{URL}/external/document", headers=headers, data=body)
    
    if response.status_code == 200:
        response_json = response.json()
        
        if "data" in response_json and "fields" in response_json["data"]:
            # print(response_json["data"])
            return response_json["data"]["fields"]
        else:
            print(f"Error: Upload fields not found in response - {response_json}")
            return None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def upload_document(bucket, fields, file_path):
    form_data = {k: v for k, v in fields.items()}
    form_data["file"] = (os.path.basename(file_path), open(file_path, 'rb'), 'application/pdf')

    m = MultipartEncoder(fields=form_data)
    
    response = requests.post(f"https://{bucket}.s3.ap-southeast-2.amazonaws.com/", data=m, headers={'Content-Type': m.content_type})
    
    if response.status_code == 204:  # 204 No Content indicates a successful S3 upload
        # print(response.json)
        print("Upload successful")
    else:
        print(f"Error: {response.status_code} - {response.text}")

if __name__ == "__main__":
    # Loop through all files in the file_category_dict and process each one
    for filepath, category in file_category_dict.items():
        # Debugging: print file and category being processed
        print(f"Processing file: {filepath}, Category: {category}")
        
        # Add document record for the current file
        document_id = add_document_record(filepath, category)
        
        if document_id:
            # Authorize upload for the created document ID
            upload_fields = authorise_upload(document_id)

            if upload_fields:
                bucket = upload_fields.get('bucket')
                if bucket:
                    # Upload the document to the specified S3 bucket
                    upload_document(bucket, upload_fields, filepath)
                else:
                    print(f"Error: No bucket found for file: {filepath}")
            else:
                print(f"Error: Upload fields not found for document ID: {document_id}")
        else:
            print(f"Error: Document ID not created for file: {filepath}")

# print(links)


{'C:\\Users\\Jiawei Qi\\Documents\\Intern\\test\\2025\\testac\\trading_entities\\Accounts Payable (800).pdf': 'Pdf', 'C:\\Users\\Jiawei Qi\\Documents\\Intern\\test\\2025\\testac\\trading_entities\\Consulting & Accounting (412).pdf': 'Pdf', 'C:\\Users\\Jiawei Qi\\Documents\\Intern\\test\\2025\\testac\\trading_entities\\GST (820).pdf': 'Pdf', 'C:\\Users\\Jiawei Qi\\Documents\\Intern\\test\\2025\\testac\\trading_entities\\Office Expenses (453).pdf': 'Pdf', 'C:\\Users\\Jiawei Qi\\Documents\\Intern\\test\\2025\\testac\\trading_entities\\Sales (200).pdf': 'Pdf', 'C:\\Users\\Jiawei Qi\\Documents\\Intern\\test\\2025\\testac\\trading_entities\\upload_bank_statements_600_bank_account.pdf': 'Pdf', 'C:\\Users\\Jiawei Qi\\Documents\\Intern\\test\\2025\\testac\\trading_entities\\upload_bank_statements_602_bank_account.pdf': 'Pdf', 'C:\\Users\\Jiawei Qi\\Documents\\Intern\\test\\2025\\testac\\trading_entities\\upload_loans_24566841.pdf': 'Pdf', 'C:\\Users\\Jiawei Qi\\Documents\\Intern\\test\\2025\\te

# 4. Generate under XLSM

In [ ]:
import xlwings as xw

#Path to the Excel file
#excel_files = 'YEAR Annual Workpapers - 20241112.xlsm'  # Update with the correct file path
#excel_name = 'YEAR Annual Workpapers.xlsm'

# Open Excel in the background
app = xw.App(visible=False)
wb = app.books.open(excel_name)
ws = wb.sheets['File Path']
# Unprotect the sheet before making any changes
ws.api.Unprotect("")
# Clear values in columns F and K starting from row 17
ws.range('F17:F' + str(ws.cells.last_cell.row)).clear_contents()
ws.range('K17:K' + str(ws.cells.last_cell.row)).clear_contents()

# Write links to Excel in column K, starting from row 2
for idx, link in enumerate(links, start=17):  # Start from row 2
    cell = ws.range(f'K{idx}')
    cell.value = link
    cell.add_hyperlink(link, text_to_display=link)  # Make it a clickable hyperlink

# Function to link column E to K via F
def link_e_to_j():
    last_row = ws.range('B' + str(ws.cells.last_cell.row)).end('up').row  # Find last row with data in column E
    for e_row in range(17, last_row + 1):
        e_value = ws.range(f'E{e_row}').value
        if e_value:
            # Find corresponding I cell for E cell
            for h_row in range(2, last_row + 1):
                if ws.range(f'I{h_row}').value == e_value:
                    # Set F cell to link to K cell of matching row
                    j_value = ws.range(f'K{h_row}').value
                    if j_value:
                        ws.range(f'F{e_row}').value = j_value
                        ws.range(f'F{e_row}').add_hyperlink(ws.range(f'K{h_row}').value, text_to_display=j_value)
                    break

# Run the link function
link_e_to_j()

# Hide columns H through K
ws.range('H:K').api.EntireColumn.Hidden = True
# Protect the sheet after making changes
ws.api.Protect("")
# Save changes to the workbook and close Excel in the background
wb.save()
# wb.close()
# app.quit()
print("Links have been added to column K and F, and columns I through K have been hidden.")


Links have been added to column K and F, and columns I through K have been hidden.


# 5. Save a copy under the local file and upload in FYI

In [6]:
# Save a copy of the workbook with the specified new name
new_name = f"2024 Annual Workpapers - {A13_value}.xlsm" 
wb.save(new_name)  # Update with your desired path for the copy
wb.close()
app.quit()

In [7]:
import os
from pathlib import Path
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder


# Default category if the file extension is not listed
default_category = 'Other'

# List to store generated links
links = []

# Define path and category for the specific file to upload
selected_text = A13_value  # Assuming A13_value is set to a valid string for file naming
new_file_path = Path(f"2024 Annual Workpapers - {selected_text}.xlsm")
file_category = file_type_mapping.get(new_file_path.suffix.lower(), default_category)

# Main script to process only the specified file
if new_file_path.is_file():  # Check if the file exists
    print(f"Processing file: {new_file_path}, Category: {file_category}")

    # Add document record for the file
    document_id = add_document_record(str(new_file_path), file_category,include_preview=False)

    if document_id:
        # Authorize upload for the created document ID
        upload_fields = authorise_upload(document_id)

        if upload_fields:
            bucket = upload_fields.get('bucket')
            if bucket:
                # Upload the document to the specified S3 bucket
                upload_document(bucket, upload_fields, new_file_path)
            else:
                print(f"Error: No bucket found for file: {new_file_path}")
        else:
            print(f"Error: Upload fields not found for document ID: {document_id}")
    else:
        print(f"Error: Document ID not created for file: {new_file_path}")

# print("Generated Links:", links)


Processing file: 2024 Annual Workpapers - testac.xlsm, Category: Spreadsheet
57a67fc9-5df6-490a-af90-67b4318c79e0
https://go.fyi.app/search/0/34493554/0/documents/57a67fc9-5df6-490a-af90-67b4318c79e0
Upload successful
